Group Memebers

*   Chirag Gupta - 10623123
*   Chukwuebuka Uwaoma - 10621864
*   Dipanshu



In [75]:
# Import necesarry libraries
import tweepy
import os
from dotenv import load_dotenv
import numpy as np
import pandas as pd
import mysql.connector
from datetime import datetime
import re
import pypyodbc

In [76]:
# Load dotenv and read secret keys and tokens
load_dotenv()
api_key = os.getenv('api_key')
api_secret_key = os.getenv('api_secret_key')
access_token = os.getenv('access_token')
access_token_secret = os.getenv('access_token_secret')

In [77]:
# Connect to twitter using tweepy API and stored tokens
auth = tweepy.OAuthHandler(api_key,api_secret_key)
auth.set_access_token(access_token,access_token_secret)
api = tweepy.API(auth)

In [78]:
# Pulls tweets based on search query and geocode location
tweetlist = []
geocode="53.1424,-7.6921,300km" # "53.3498053,-6.2603097,25km"
filter =  '-filter:retweets AND -filter:replies'
query = 'housing'
tweets = tweepy.Cursor(api.search_tweets, q=query+filter, geocode=geocode, lang='en', tweet_mode="extended", count=20).items(20)

# adds tweets to a dataframe
column_names = ["id", "created_at", "username", "tweet", "retweet_count", "favorite_count", "user_location"]
df = pd.DataFrame(columns=column_names)

for tweet in tweets:
    #create a new row object
    new_row = pd.Series({
        "id": tweet.id, 
        "created_at":tweet.created_at,
        "username": tweet.user.screen_name, 
        "tweet": tweet.full_text, 
        "retweet_count": tweet.retweet_count, 
        "favorite_count": tweet.favorite_count,
        "user_location":tweet.user.location,
    })
    
    # get the index of the last row
    last_row_index = len(df)
    # add the new row to the DataFrame using .iloc[]
    df.loc[last_row_index] = new_row
df.head()
    
# Pull tweet text, id, created_at_date,  username, retweets, likes saved to temporary dataframe - Virgil - DONE

,id,created_at,username,tweet,retweet_count,favorite_count,user_location
0,1648299993034268674,2023-04-18 12:18:36+00:00,kevosullivan07,Listening to @RTENewsAtOne and we've got a sur...,0,0,Co. Cork
1,1648287760627761153,2023-04-18 11:30:00+00:00,CouncilJournal,"Over 10,000 Social Homes Delivered in 2022 Wit...",0,0,"Dublin City, Ireland"
2,1648283256234598406,2023-04-18 11:12:06+00:00,LimerickCouncil,Are you a private developer or small builder? ...,1,1,"Limerick, Ireland"
3,1648282983994892289,2023-04-18 11:11:01+00:00,LimerickCouncil,Information for Tenants in Limerick who Receiv...,0,0,"Limerick, Ireland"
4,1648281977311703041,2023-04-18 11:07:01+00:00,Dublin_Simon,It is unacceptable that anyone should have to ...,1,0,Dublin


In [79]:
# Clean the data
    ## clean text 
    ## date_time in right format eg dd/mm/yyyy - Chirag

# connect to a external DB upload cleaned data
    ## make sure tweets are unique by id - Dipanshu

# pull from database assign to dataframe - DP 
    # tokenize - virgil
    # lematize  - Dipanshu
    # visualise word cloud - Chirag

In [80]:
# Pre-processing the data

# Formatting the date to dd/mm/YYYY format
def formatDate(timestamp):
    # As the timestamp retrieved from the api is already in DatetimeIndex form, simply running string format function 
    # of the datetime library and passing the desired format
    formatted_date = timestamp.strftime("%d/%m/%Y")
    return formatted_date

# Getting only the words and numbers from the tweet that contains links, special symbols, emojis, etc.
def cleanTweet(tweet):
    # Converting the string to lowercase and replacing the ' with blank
    removed_apostrophe = tweet.lower().replace("'","")
    # Using regex pattern to replace the links, <html> tags and &amp; symbol (seen frequently in the tweets) with blank
    removed_links_tags_frequentsymbol = re.sub("(https?://\S+)|(<[\w]+>)|(&amp;)","",removed_apostrophe)
    # This regex pattern returns a list that contains all the words and numbers in the tweet
    cleaned_tweet = re.findall("([a-z0-9]+)", removed_links_tags_frequentsymbol)
    cleaned_tweet = " ".join(cleaned_tweet)
    return cleaned_tweet

# Applying respective functions to both columns in the dataset
df["created_at"] = df["created_at"].apply(lambda x: formatDate(x))
df["tweet"] = df["tweet"].apply(lambda x: cleanTweet(x))
df.head()

,id,created_at,username,tweet,retweet_count,favorite_count,user_location
0,1648299993034268674,18/04/2023,kevosullivan07,listening to rtenewsatone and weve got a surpl...,0,0,Co. Cork
1,1648287760627761153,18/04/2023,CouncilJournal,over 10 000 social homes delivered in 2022 wit...,0,0,"Dublin City, Ireland"
2,1648283256234598406,18/04/2023,LimerickCouncil,are you a private developer or small builder o...,1,1,"Limerick, Ireland"
3,1648282983994892289,18/04/2023,LimerickCouncil,information for tenants in limerick who receiv...,0,0,"Limerick, Ireland"
4,1648281977311703041,18/04/2023,Dublin_Simon,it is unacceptable that anyone should have to ...,1,0,Dublin


In [81]:
connection = mysql.connector.connect(host='localhost',
                                         database='ca',
                                         user='root',
                                         password='toor')

In [82]:
# SQL query to create a table and inserting the columns name in it.
mySql_insert_query = 'CREATE TABLE if not exists api (id int(22) primary key ,created_at DATE,username varchar(255),tweet TEXT(50000),retweet_count int(255),favorite_count int(255),user_location varchar(255));'

In [83]:
# Create a cursor object "cursor" to execute MySQL queries with.
# The "buffered=True" parameter sets up the cursor to buffer the results, which can be useful for large queries.
cursor = connection.cursor(buffered=True)

# Use the cursor's "execute()" method to execute the MySQL query specified in the "mySql_insert_query" variable.
cursor.execute(mySql_insert_query)

# Commit the changes made to the MySQL database using the connection's "commit()" method.
connection.commit()

In [84]:
# Use a for loop to iterate over the rows of the Pandas DataFrame "df".

for i in range(df.shape[0]):
    # Initialize an empty string "s" to store the column names.
    s=""
    # Use another for loop to iterate over the columns of the DataFrame and concatenate the column names to the "s" string.
    for j in df.columns:
        s=s+j+","
    # Remove the last comma from the "s" string.
    s=s[:-1]

    # Initialize an empty string "t" to store the values in the current row.
    t=""
    # Use another for loop to iterate over the columns of the DataFrame and concatenate the values to the "t" string, 
    # converting them to strings and enclosing them in quotes.
    for k in df.columns:
        t=t+"'"+str(df[k].iloc[i])+"',"
    # Remove the last comma from the "t" string.
    t=t[:-1]
    
    # Create a MySQL query "command" to insert the current row into the "api" table.
    # The column names are specified by the "s" string, and the values are specified by the "t" string.
    command="INSERT INTO api ("+s+") VALUES ("+t+");"
    # Print the MySQL query to the console.
    print(command)
    
    # Use a try-except block to execute the MySQL query using the cursor's "execute()" method.
    # If an error occurs, ignore it and continue to the next row.
    try:
        cursor.execute(command)
    except:
        pass

# Commit the changes made to the MySQL database using the connection's "commit()" method.
connection.commit()


INSERT INTO api (id,created_at,username,tweet,retweet_count,favorite_count,user_location) VALUES ('1648299993034268674','18/04/2023','kevosullivan07','listening to rtenewsatone and weve got a surplus of 8 billion quid housing figures are dire cuh are under savage pressure and weve 8bn unallocated and projected 10bn next year','0','0','Co. Cork ');
INSERT INTO api (id,created_at,username,tweet,retweet_count,favorite_count,user_location) VALUES ('1648287760627761153','18/04/2023','CouncilJournal','over 10 000 social homes delivered in 2022 with 43 increase in new build social homes read more here social housing housingforall ireland darraghobrientd depthousingirl','0','0','Dublin City, Ireland');
INSERT INTO api (id,created_at,username,tweet,retweet_count,favorite_count,user_location) VALUES ('1648283256234598406','18/04/2023','LimerickCouncil','are you a private developer or small builder our turnkey acquisition campaign could kick start your development expressions of interest invited 

In [85]:
# Data retrieve from DB and insert into pd df

# Define a MySQL query "sql_query" to select all columns from the "api" table.
sql_query = "SELECT * FROM api"

# Use the cursor's "execute()" method to execute the MySQL query.
cursor.execute(sql_query)

# Use the cursor's "fetchall()" method to fetch all the rows returned by the MySQL query.
result = cursor.fetchall()

# Convert the fetched rows into a Pandas DataFrame "data" using the "pd.DataFrame()" function.
data=pd.DataFrame(result)

# Rename the columns of the DataFrame using the "columns" attribute of the DataFrame.
data.columns=["id", "created_at", "username", "tweet", "retweet_count", "favorite_count", "user_location"]

# Print the DataFrame "data" to the console.
data



,id,created_at,username,tweet,retweet_count,favorite_count,user_location
0,1647634340082335750,16/04/2023,UrbanArchCon,6 collective housing interventions that rethin...,0,0,"South Dublin, Ireland"
1,1647635355229724672,16/04/2023,luke_oriordan,minister for housing must explain department u...,0,2,"Glasnevin, Dublin"
2,1647641119440744456,16/04/2023,RoryHearneGaffs,irelands housing crisis generation catastrophe...,35,117,Dublin
3,1647655780730929152,16/04/2023,oceanclub,quite a gulf between this anon fine gael tds s...,2,8,"Dublin, Ireland"
4,1647656641074982916,16/04/2023,john_e_horan,really enjoyed this and feels true ireland now...,0,0,"South Dublin, Ireland"
5,1647660755959918592,16/04/2023,picturesby_JOE,kiosk clontarf promenade dublin corporation ho...,1,4,"Dublin City, Ireland"
6,1647676862787715074,16/04/2023,JoeSeward1,im starting to think the three toughest jobs o...,0,0,"Cork City, Ireland"
7,1647678790468812802,16/04/2023,MaryCallaghanSD,100k vacant homes in ireland in the middle of ...,0,0,"Dublin City, Ireland"
8,1647683852213669891,16/04/2023,kevintoconnor,fine gael is a very chilling party its tds are...,15,33,Dublin
9,1647692554261999623,16/04/2023,winkenswexford,tract housing copy paste houses housingcrisis ...,0,0,"Wexford, Ireland"
